# 1. Problemas mal condicionados

In [1]:
v1=[-1;1];
v2=[-2;2];

In [2]:
A=[v1 v2]

2×2 Matrix{Int64}:
 -1  -2
  1   2

El número de condicionamiento de una matriz indica cuando la solución de un sistema lineal podría sufrir cambios grandes a partir de cambios pequeños en las entradas y por lo tanto sería inexacto. Al mismo tiempo, peuqeños errores de redondeo pueden causar errores significativos al resolver un problema mal condicionado.

El número de condicionamiento se define como el producto de la norma $L_p$ de la matriz y la norma de la matriz inversa:

$cond(A) = \|A\|_p \times \|A^{-1}\|_p$

In [16]:
using LinearAlgebra;

cond(A)

Inf

In [12]:
norm(A,Inf)

2.0

Al mismo tiempo, sabemos que la inversa de una matriz no está definida si el determinante de esta es cero.

In [14]:
det(A)

0.0

In [17]:
using Random;

rng = MersenneTwister(1);

v3 = rand(rng,2);
v4 = rand(rng,2);

In [18]:
A=[v3 v4]

2×2 Matrix{Float64}:
 0.236033  0.312707
 0.346517  0.00790928

In [19]:
cond(A)

2.091388793896744

In [20]:
det(A)

-0.10649143036437071

In [21]:
A*inv(A)

2×2 Matrix{Float64}:
 1.0          -1.11022e-16
 3.46945e-18   1.0

Evaluar el número de condicion de la matriz $X$ y el producto $X \times X^-1$ para :

$X=\begin{bmatrix}
0.12065  & 0.98775\\
 0.1232  & 0.98755\\
\end{bmatrix}$

In [23]:
X=[0.12065 0.98775;
   0.1232  0.98755]

2×2 Matrix{Float64}:
 0.12065  0.98775
 0.1232   0.98755

In [30]:
rank(X)

2

In [31]:
cond(X)

778.8911496416749

In [32]:
norm(X)*norm(inv(X))

778.8924335181275

In [33]:
det(X)

-0.0025428925000000064

In [34]:
X*inv(X)

2×2 Matrix{Float64}:
 1.0          0.0
 7.10543e-15  1.0

# 2. Eliminación de Gauss

$Ax=y$

In [35]:
A=[-0.04 0.04 0.12 ; 0.56 -1.56 0.32 ; -0.24 1.24 -0.28];
y=[3; 1 ;0];

El rango de una matriz nos entrega la cantidad máxima de columnas columnas que son linealmente independientes

In [36]:
using LinearAlgebra;

rank(A)

3

In [37]:
cond(A)

19.304418511266785

In [38]:
inv(A)

3×3 Matrix{Float64}:
 1.0  4.0  5.0
 2.0  1.0  2.0
 8.0  1.0  1.0

In [39]:
det(A)

0.04

creamos la matriz ampliada $[A y]$

In [40]:
A_c=[A y]

3×4 Matrix{Float64}:
 -0.04   0.04   0.12  3.0
  0.56  -1.56   0.32  1.0
 -0.24   1.24  -0.28  0.0

1.1) Tomamos la primera fila de la matriz aumentada como pivote y creamos el multiplicador $m_{2,1}$. Luego restamos la segunda fila con el pivote y el multipicador. 

In [42]:
P=A_c[1,:];
m21=A_c[2,1]/A_c[1,1];
A_c[2,:]-=m21*P;

In [44]:
A

3×3 Matrix{Float64}:
 -0.04   0.04   0.12
  0.56  -1.56   0.32
 -0.24   1.24  -0.28

In [43]:
A_c

3×4 Matrix{Float64}:
 -0.04   0.04   0.12   3.0
  0.0   -1.0    2.0   43.0
 -0.24   1.24  -0.28   0.0

2.2) Creamos el multiplicador $m_{3,1}$. Luego restamos la tercera fila con el pivote y el multipicador. 

In [45]:
m31=A_c[3,1]/A_c[1,1];
A_c[3,:]-=m31*P;

In [46]:
A_c

3×4 Matrix{Float64}:
 -0.04   0.04   0.12    3.0
  0.0   -1.0    2.0    43.0
  0.0    1.0   -1.0   -18.0

1.3) Tomamos la segunda fila de la matriz aumentada como pivote y creamos el multiplicador $m_{3,2}$. Luego restamos la tercera fila con el pivote y el multipicador. 

In [47]:
P=A_c[2,:]
m32=A_c[3,2]/A_c[2,2];
A_c[3,:]-=m32*P;

In [48]:
A_c

3×4 Matrix{Float64}:
 -0.04   0.04  0.12   3.0
  0.0   -1.0   2.0   43.0
  0.0    0.0   1.0   25.0

1.4) Resolvemos la tercera ecuación para $x_3=25$. Luego sustuimos en $-x_2+2*25=43$ y obtenemos $x_2=7$.

Finalmente sustituimos $x_1$ y $x_2$ en la primera ecuación $-004*x_1+0.004*x_2+0.12*x_3=3$ y obtenemos $x_1=7$.

In [ ]:
function gaussian_elimination(A,y)
    A_c=[A y]
    n,m=size(A_c)
    for i=1:(n-1)
        for j=i+1:n
            P=A_c[i,:]
            mji=A_c[j,i]/P[i];
            A_c[j,:]-=mji*P;
        end
    end
    return A_c
end 

In [49]:
# Algoritmo 1.1 VMLS https://web.stanford.edu/~boyd/vmls/

function back_subst(R,b)
    n = length(b)
    x = zeros(n)
    for i=n:-1:1
        x[i] = (b[i] - R[i,i+1:n]'*x[i+1:n]) / R[i,i]
    end
    return x
end;

In [50]:
x_gauss=back_subst(A_c[:,1:3],A_c[:,4])

3-element Vector{Float64}:
  7.000000000000028
  7.000000000000007
 25.000000000000007

En Julia podemos resolver directamente el sistema de ecuaciones mediante el operador \ 

In [51]:
x_ls=A\y

3-element Vector{Float64}:
  7.000000000000005
  7.000000000000002
 25.0

In [52]:
inv(A)*y

3-element Vector{Float64}:
  7.000000000000003
  7.000000000000002
 25.0

# 2) Ejercicios 

2.1) Resolver el siguiente sistema de ecuaciones lineales usando el método de eliminación de Gauss: $\boldsymbol{A}\boldsymbol{x}=\boldsymbol{y}$ para la matriz de Hilbert de tamaño $3 \times 3$:

$\boldsymbol{A}=\begin{bmatrix} 1/2 & 1/3 & 1/4 \\
                                1/3 & 1/4 & 1/5 \\
                                1/4 & 1/5 & 1/6 \\
                \end{bmatrix}$, 
$\boldsymbol{y}=\begin{bmatrix} 1 \\
                                1 \\
                                1 \\
                \end{bmatrix}$

2.2) Comprobar los resultados con los valores obtenidos.

2.3) Una forma de evaluar la existencia de la solución de un sistema de ecuaciones es a través del número de condicionamiento:

$cond(A)=||A||\times ||A^{-1}|| $, donde $||\cdot||$ es la norma de la matriz. Revise el determinante y el número de condicionamiento de la matriz de ejercicio 2.1.

2.4) Construya una matriz de Hilbert de tamaño $20 \times 20$ y repita el procedimiento anterior. ¿La matriz resultante es invertible? ¿Cuanto cambia el número de condicionamiento.? 

$A=[\frac{1}{i+j}],i,j=1:20$

In [55]:
function hilbert(n::Int64)::Matrix
    A=ones(n,n)
    for i=1:n
        for j=1:n
            A[i,j]=1/(i+j)
        end
    end
    return A 
end

hilbert (generic function with 1 method)

In [56]:
A=hilbert(3)

3×3 Matrix{Float64}:
 0.5       0.333333  0.25
 0.333333  0.25      0.2
 0.25      0.2       0.166667

2.5) Resolver el siguiente sistema de ecuaciones lineales mediante eliminación de Gauss:

$\boldsymbol{A}=\begin{bmatrix} 2 & 4 & 6 \\
                                2 & 0 & 2 \\
                                6 & 8 &24 \\
                \end{bmatrix}$, 
$\boldsymbol{y}=\begin{bmatrix} 1 \\
                                1 \\
                                1 \\
                \end{bmatrix}$